# Notebook Memory Networks - Question Answering

### The dataset can be downloaded - https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz

In [7]:
# import the necessary packages

from keras.layers import Input
from keras.layers.core import Dense, Activation, Dropout, Permute
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.models import Model

import numpy as np
import itertools
import matplotlib.pyplot as plt

%matplotlib inline

### Read the files to create train and test data

In [27]:
TRAIN_FILE = "/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/datasets/FB_BABL_Ques_Answering/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt"
TEST_FILE = "/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/datasets/FB_BABL_Ques_Answering/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt"

stories = []
ques = []
answers = []
def parseFile(infile):
    story = []
    # This parses each line and creates the stories, questions and answers tuple
    for line in open(infile):
        content = line.split(" ", 1)
        if "\t" in line:
            stor_ques_ans = line.split("\t")
            ques.append(stor_ques_ans[0])
            answers.append(stor_ques_ans[1])
            stories.append(story)
            story = []
        else:
            story.append(content[1])
    print(ques)
    return stories, ques, answers
    
train_data = parseFile(TRAIN_FILE)
test_data = parseFile(TEST_FILE)

['3 Where is Mary? ', '6 Where is Daniel? ', '9 Where is Daniel? ', '12 Where is Daniel? ', '15 Where is Sandra? ', '3 Where is Sandra? ', '6 Where is Sandra? ', '9 Where is Sandra? ', '12 Where is John? ', '15 Where is Daniel? ', '3 Where is John? ', '6 Where is Mary? ', '9 Where is John? ', '12 Where is John? ', '15 Where is Sandra? ', '3 Where is Daniel? ', '6 Where is Sandra? ', '9 Where is Sandra? ', '12 Where is Sandra? ', '15 Where is Daniel? ', '3 Where is John? ', '6 Where is Mary? ', '9 Where is John? ', '12 Where is Sandra? ', '15 Where is John? ', '3 Where is Daniel? ', '6 Where is John? ', '9 Where is Daniel? ', '12 Where is John? ', '15 Where is Daniel? ', '3 Where is Mary? ', '6 Where is Sandra? ', '9 Where is Daniel? ', '12 Where is John? ', '15 Where is Daniel? ', '3 Where is Mary? ', '6 Where is John? ', '9 Where is Daniel? ', '12 Where is John? ', '15 Where is Sandra? ', '3 Where is Daniel? ', '6 Where is Daniel? ', '9 Where is Daniel? ', '12 Where is Sandra? ', '15 

In [21]:
print(train_data[2])
print(" ")
print(test_data[2])

['1\n', '4\n', '4\n', '11\n', '8\n', '2\n', '2\n', '2\n', '8\n', '11\n', '2\n', '1\n', '7\n', '7\n', '10\n', '2\n', '5\n', '8\n', '8\n', '14\n', '2\n', '1\n', '5\n', '7\n', '13\n', '2\n', '5\n', '8\n', '10\n', '8\n', '2\n', '5\n', '7\n', '11\n', '14\n', '1\n', '2\n', '8\n', '7\n', '14\n', '2\n', '5\n', '5\n', '8\n', '11\n', '2\n', '2\n', '8\n', '11\n', '10\n', '2\n', '5\n', '5\n', '8\n', '8\n', '1\n', '5\n', '4\n', '11\n', '10\n', '2\n', '4\n', '5\n', '5\n', '13\n', '1\n', '5\n', '4\n', '10\n', '14\n', '2\n', '5\n', '7\n', '11\n', '13\n', '1\n', '5\n', '7\n', '8\n', '13\n', '2\n', '4\n', '8\n', '10\n', '11\n', '1\n', '5\n', '5\n', '8\n', '13\n', '1\n', '1\n', '7\n', '11\n', '14\n', '2\n', '5\n', '5\n', '11\n', '14\n', '1\n', '5\n', '8\n', '10\n', '10\n', '2\n', '4\n', '8\n', '7\n', '14\n', '2\n', '2\n', '7\n', '10\n', '13\n', '1\n', '5\n', '8\n', '11\n', '11\n', '1\n', '5\n', '8\n', '11\n', '11\n', '1\n', '5\n', '7\n', '10\n', '13\n', '2\n', '5\n', '2\n', '10\n', '13\n', '1\n', '5\n', 

### Build Vocab from traina and test data

In [ ]:
# this method returns the word2index and index2word mappings 

def buildVocab(train_data, test_data):
    for 

### Get Maximum Lengths for the story and Question

### Vectorize the Inputs and Pad the sequences

### Define the Model Architecture